## Post Assistant


In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from tavily import TavilyClient

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
# setting up groq api key
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [4]:
# chat set up
model = ChatGroq(temperature=0, model_name="llama3-8b-8192")

In [5]:
# testing the model
print(model.invoke("Hello, how are you?").content)

I'm just an AI, I don't have feelings or emotions like humans do, so I don't have a good or bad day. I'm just here to help answer your questions and provide information to the best of my ability. How can I assist you today?


## Agent - 1: Post rewritter/enhancer


#### Input:

- Instruction
- Post


#### Process.

- task: task
- plan: Plan on making it better
- draft: The initial post draft
- critique: Analyzing the generated post to be sure it good.
- content: The post content
- revision_number: The number of revisions made
- max_revision : The max number of revisions made while processing the post


In [6]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [7]:
PLAN_PROMPT = """You are an expert post creator tasked with writing and crafting a high level outline on how to improve a post. \
You observe the user instruction and initial post, then create an outline on it's improvement. \
Understand the user's instruction and the post, then create a plan on how to improve the post. \
Give an outline of the post improvement along with any relevant notes or instructions for the sections.\
Remember to observe the paragraph structure and the content of the post. Add all relevant information to the outline.
"""

In [8]:
WRITER_PROMPT = """You are a post assistant tasked with writing, crafting, improving and enhancing posts.\
Generate the best content possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [9]:
REFLECTION_PROMPT = """You are a teacher grading an post submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for tone, length, depth, style, etc."""

In [10]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when composing the a better post. Generate a list of search queries that will gather \
any relevant information. Only generate 2 queries max. it should be relevant to the post and the user's instruction.\
If there's no need for a query, respond with "None".    
"""


In [11]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [12]:
from langchain_core.pydantic_v1 import BaseModel
import time

class Queries(BaseModel):
    queries: List[str]

In [13]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [14]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [15]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [16]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [17]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [18]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [19]:
builder = StateGraph(AgentState)

In [20]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [21]:
builder.set_entry_point("planner")

In [22]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [23]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [24]:
graph = builder.compile(checkpointer=memory)

In [25]:
from IPython.display import Image

# Image(graph.get_graph().draw_png())

In [26]:
thread = {"configurable": {"thread_id": "1"}}
post = """
📊 Exploring Data Science 📊

Data science combines programming, statistics, and domain expertise to extract insights from data. It involves collecting, cleaning, analyzing, and visualizing data to make informed decisions.

Key steps:

Collect Data 🗂️
Clean Data 🧹
Analyze Data 📉
Visualize Data 📊
Data science is transforming industries by uncovering patterns and driving innovation. It's a field full of opportunities and growth.

"""
instruction = """
Engage Readers: End with a question like, "How do you see data science impacting your daily life?"
Include Links: Provide a link to a beginner's guide to data science for more information.
"""

# combining the instruction and the post
task = f"{instruction}\n\n{post}"

# starting the thread
for s in graph.stream({
    'task': task,
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': 'Here is a high-level outline on how to improve the post:\n\nI. Introduction\n\n* Start with a hook to grab the reader\'s attention\n* Introduce the topic of data science and its importance\n* Thesis statement: Data science is a powerful tool that can transform industries and improve our daily lives\n\nII. What is Data Science?\n\n* Define data science and its components (programming, statistics, and domain expertise)\n* Explain the process of data science (collecting, cleaning, analyzing, and visualizing data)\n* Provide examples of how data science is used in different industries\n\nIII. Benefits of Data Science\n\n* Discuss the benefits of data science, such as:\n\t+ Uncovering patterns and trends\n\t+ Driving innovation and growth\n\t+ Improving decision-making\n* Provide examples of how data science is transforming industries\n\nIV. How Data Science Impacts Daily Life\n\n* End with a question that encourages readers to think about how data science impacts thei

In [31]:
print(s['generate']['draft'])

Here's a rewritten version of the post that incorporates the outline and suggestions:

📊 Exploring Data Science 📊

**The Power of Data Science**

Have you ever wondered how your favorite music streaming service knows exactly what songs to recommend to you? Or how your favorite TV show knows exactly what you'll love to watch next? It's all thanks to the magic of data science! 🧙‍♀️

Data science is a powerful tool that combines programming, statistics, and domain expertise to extract insights from data. It's a field that's transforming industries and improving our daily lives. But what exactly is data science, and how does it work?

**What is Data Science?**

Data science is a process that involves collecting, cleaning, analyzing, and visualizing data to make informed decisions. It's a field that's used in various industries, from healthcare to finance, and even entertainment. For example, music streaming services use data science to recommend songs based on your listening habits. 🎵

**B

In [28]:
graph

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<task,plan,draft,critique,content,revision_number,max_revisions>(recurse=True, writes=[ChannelWriteEntry(channel='task', value=<object object at 0x00000254EE5D6700>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='plan', value=<object object at 0x00000254EE5D6700>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='draft', value=<object object at 0x00000254EE5D6700>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='critique', value=<object object at 0x00000254EE5D6700>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='content', value=<object object at 0x00000254EE5D6700>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='revision_number', value=<object object at 0x0

In [32]:
# creating a function
def run_graph(task: str, max_revisions: int = 2, thread: dict = {}):
    for s in graph.stream({
    'task': task,
    "max_revisions": 2,
    "revision_number": 1,
    }, thread):
        print(s)
        
    # return the final state
    return s['generate']['draft']


In [33]:
# testing the function
thread = {"configurable": {"thread_id": "1"}}
new_post = run_graph(task, thread=thread)

{'planner': {'plan': 'Here is a high-level outline on how to improve the post:\n\nI. Introduction\n\n* Start with a hook to grab the reader\'s attention\n* Introduce the topic of data science and its importance\n* Thesis statement: Data science is a powerful tool that can transform industries and improve our daily lives\n\nII. What is Data Science?\n\n* Define data science and its components (programming, statistics, and domain expertise)\n* Explain the process of data science (collecting, cleaning, analyzing, and visualizing data)\n* Provide examples of how data science is used in different industries\n\nIII. Benefits of Data Science\n\n* Discuss the benefits of data science, such as:\n\t+ Uncovering patterns and trends\n\t+ Driving innovation and growth\n\t+ Improving decision-making\n* Provide examples of how data science is transforming industries\n\nIV. How Data Science Impacts Daily Life\n\n* End with a question that encourages readers to think about how data science impacts thei

KeyError: 'state'